# Prática: Texturas + Model,View,Projection


### Primeiro, vamos importar as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [1]:
import glfw
from OpenGL.GL import * # type: ignore
import numpy as np
import glm
import math
from PIL import Image

glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
largura = 1920
altura = 1080
window = glfw.create_window(largura, altura, "Malhas e Texturas", None, None)
glfw.make_context_current(window)

In [2]:
vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        varying vec2 out_texture;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
        }
        """
fragment_code = """
        uniform vec4 color;
        varying vec2 out_texture;
        uniform sampler2D samplerTexture;
        
        void main(){
            vec4 texture = texture2D(samplerTexture, out_texture);
            gl_FragColor = texture;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [3]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")


# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Carregando Modelos (vértices e texturas) a partir de Arquivos

A função abaixo carrega modelos a partir de arquivos no formato WaveFront.


Para saber mais sobre o modelo, acesse: https://en.wikipedia.org/wiki/Wavefront_.obj_file


Nos slides e vídeo-aula da Aula 11 - Parte 1, nós descrevemos o funcionamento desse formato.

In [4]:
def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """
    objects = {}
    vertices = []
    texture_coords = []
    faces = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(filename, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.split() # quebra a linha por espaço
        if not values: continue


        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])


        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces

    return model



In [5]:
glHint(GL_LINE_SMOOTH_HINT, GL_DONT_CARE)
glEnable( GL_BLEND )
glBlendFunc( GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA )
glEnable(GL_LINE_SMOOTH)
glEnable(GL_TEXTURE_2D)
qtd_texturas = 2
textures = glGenTextures(qtd_texturas)

def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    print(img_textura,img.mode)
    img_width = img.size[0]
    img_height = img.size[1]
    #image_data = img.tobytes("raw", "RGB", 0, -1)
    image_data = img.convert("RGBA").tobytes("raw", "RGBA",0,-1)

    #image_data = np.array(list(img.getdata()), np.uint8)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, img_width, img_height, 0, GL_RGBA, GL_UNSIGNED_BYTE, image_data)

### A lista abaixo armazena todos os vertices carregados dos arquivos

In [6]:
vertices_list = []    
textures_coord_list = []

### Vamos carregar cada modelo e definir funções para desenhá-los

In [7]:
modelo = load_model_from_file('lua/lua.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cube.obj. Vertice inicial:',len(vertices_list))
a = len(vertices_list)
faces_visited = []

for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], ' vertice inicial =', len(vertices_list))
        faces_visited.append(face[2])
    
    vertices = [modelo['vertices'][i - 1] for i in face[0]]
    textures = [modelo['texture'][i - 1] for i in face[1]]

    if len(face[0]) == 3:  # Triângulo
        vertices_list.extend(vertices)
        textures_coord_list.extend(textures)
    elif len(face[0]) == 4:  # Quadrilátero
        # Divisão em dois triângulos
        vertices_list.extend([vertices[0], vertices[1], vertices[2], vertices[0], vertices[2], vertices[3]])
        textures_coord_list.extend([textures[0], textures[1], textures[2], textures[0], textures[2], textures[3]])


print('total de vertices:',len(vertices_list)-a)

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(0,'lua/lua.jpg')


Processando modelo cube.obj. Vertice inicial: 0
Material.002  vertice inicial = 0
total de vertices: 18720
lua/lua.jpg L


In [8]:
modelo = load_model_from_file('caixa/caixa.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo spiderman.obj. Vertice inicial:',len(vertices_list))
a = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )

print("total de vertices:",len(vertices_list)-a)

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(1,'caixa/caixa.jpg')


Processando modelo spiderman.obj. Vertice inicial: 18720
total de vertices: 36
caixa/caixa.jpg RGB


In [9]:
modelo = load_model_from_file('nave/nave.obj')

a = len(vertices_list)
### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cube.obj. Vertice inicial:',len(vertices_list))
faces_visited = []

for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], ' vertice inicial =', len(vertices_list))
        faces_visited.append(face[2])
    
    vertices = [modelo['vertices'][i - 1] for i in face[0]]
    textures = [modelo['texture'][i - 1] for i in face[1]]

    if len(face[0]) == 3:  # Triângulo
        vertices_list.extend(vertices)
        textures_coord_list.extend(textures)
    elif len(face[0]) == 4:  # Quadrilátero
        # Divisão em dois triângulos
        vertices_list.extend([vertices[0], vertices[1], vertices[2], vertices[0], vertices[2], vertices[3]])
        textures_coord_list.extend([textures[0], textures[1], textures[2], textures[0], textures[2], textures[3]])

print(len(vertices_list)-a)
### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(2,'nave/Spaceship_Ferry_diffuse.jpg')


Processando modelo cube.obj. Vertice inicial: 18756
Spaceship_Ferry  vertice inicial = 18756
325632
nave/Spaceship_Ferry_diffuse.jpg RGB


In [10]:
modelo = load_model_from_file('terra/terra.obj')

a = len(vertices_list)
### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cube.obj. Vertice inicial:',len(vertices_list))
a = len(vertices_list)
faces_visited = []

for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], ' vertice inicial =', len(vertices_list))
        faces_visited.append(face[2])
    
    vertices = [modelo['vertices'][i - 1] for i in face[0]]
    textures = [modelo['texture'][i - 1] for i in face[1]]

    if len(face[0]) == 3:  # Triângulo
        vertices_list.extend(vertices)
        textures_coord_list.extend(textures)
    elif len(face[0]) == 4:  # Quadrilátero
        # Divisão em dois triângulos
        vertices_list.extend([vertices[0], vertices[1], vertices[2], vertices[0], vertices[2], vertices[3]])
        textures_coord_list.extend([textures[0], textures[1], textures[2], textures[0], textures[2], textures[3]])


### inserindo coordenadas de textura do modelo no vetor de texturas
print('tamanho:',len(textures_coord_list)-a)

### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(3,'terra/earth albedo.jpg')


Processando modelo cube.obj. Vertice inicial: 344388
earth  vertice inicial = 344388
tamanho: 11904
terra/earth albedo.jpg RGB


In [11]:
modelo = load_model_from_file('sol/sol.obj')

a = len(vertices_list)
### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cube.obj. Vertice inicial:',len(vertices_list))
a = len(vertices_list)
faces_visited = []

for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], ' vertice inicial =', len(vertices_list))
        faces_visited.append(face[2])
    
    vertices = [modelo['vertices'][i - 1] for i in face[0]]
    textures = [modelo['texture'][i - 1] for i in face[1]]

    if len(face[0]) == 3:  # Triângulo
        vertices_list.extend(vertices)
        textures_coord_list.extend(textures)
    elif len(face[0]) == 4:  # Quadrilátero
        # Divisão em dois triângulos
        vertices_list.extend([vertices[0], vertices[1], vertices[2], vertices[0], vertices[2], vertices[3]])
        textures_coord_list.extend([textures[0], textures[1], textures[2], textures[0], textures[2], textures[3]])


### inserindo coordenadas de textura do modelo no vetor de texturas
print('tamanho:',len(textures_coord_list)-a)

### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(4,'sol/sol.jpg')

Processando modelo cube.obj. Vertice inicial: 356292
13913_Sun  vertice inicial = 356292
tamanho: 47616
sol/sol.jpg RGB


In [12]:
modelo = load_model_from_file('volante/volante.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cube.obj. Vertice inicial:',len(vertices_list))
a = len(vertices_list)
faces_visited = []

for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], ' vertice inicial =', len(vertices_list))
        faces_visited.append(face[2])
    
    vertices = [modelo['vertices'][i - 1] for i in face[0]]
    textures = [modelo['texture'][i - 1] for i in face[1]]

    if len(face[0]) == 3:  # Triângulo
        vertices_list.extend(vertices)
        textures_coord_list.extend(textures)
    elif len(face[0]) == 4:  # Quadrilátero
        # Divisão em dois triângulos
        vertices_list.extend([vertices[0], vertices[1], vertices[2], vertices[0], vertices[2], vertices[3]])
        textures_coord_list.extend([textures[0], textures[1], textures[2], textures[0], textures[2], textures[3]])


print('total de vertices:',len(vertices_list)-a)

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(5,'volante/madeira.jpg')


Processando modelo cube.obj. Vertice inicial: 403908
None  vertice inicial = 403908
total de vertices: 176328
volante/madeira.jpg RGB


In [13]:
modelo = load_model_from_file('cama/cama.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cube.obj. Vertice inicial:',len(vertices_list))
a = len(vertices_list)
faces_visited = []

for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], ' vertice inicial =', len(vertices_list))
        faces_visited.append(face[2])
    
    vertices = [modelo['vertices'][i - 1] for i in face[0]]
    textures = [modelo['texture'][i - 1] for i in face[1]]

    if len(face[0]) == 3:  # Triângulo
        vertices_list.extend(vertices)
        textures_coord_list.extend(textures)
    elif len(face[0]) == 4:  # Quadrilátero
        # Divisão em dois triângulos
        vertices_list.extend([vertices[0], vertices[1], vertices[2], vertices[0], vertices[2], vertices[3]])
        textures_coord_list.extend([textures[0], textures[1], textures[2], textures[0], textures[2], textures[3]])


print('total de vertices:',len(vertices_list)-a)

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(6,'cama/cama.jpg')


Processando modelo cube.obj. Vertice inicial: 580236
Sleeping_Bag_Unrolled_V1_SG  vertice inicial = 580236
total de vertices: 27264
cama/cama.jpg RGB


In [14]:
modelo = load_model_from_file('cadeira/cadeira.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cube.obj. Vertice inicial:',len(vertices_list))
a = len(vertices_list)
faces_visited = []

for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], ' vertice inicial =', len(vertices_list))
        faces_visited.append(face[2])
    
    vertices = [modelo['vertices'][i - 1] for i in face[0]]
    textures = [modelo['texture'][i - 1] for i in face[1]]

    if len(face[0]) == 3:  # Triângulo
        vertices_list.extend(vertices)
        textures_coord_list.extend(textures)
    elif len(face[0]) == 4:  # Quadrilátero
        # Divisão em dois triângulos
        vertices_list.extend([vertices[0], vertices[1], vertices[2], vertices[0], vertices[2], vertices[3]])
        textures_coord_list.extend([textures[0], textures[1], textures[2], textures[0], textures[2], textures[3]])


print('total de vertices:',len(vertices_list)-a)

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(7,'cadeira/cadeira.jpg')


Processando modelo cube.obj. Vertice inicial: 607500
13494_Folding_Chairs  vertice inicial = 607500
total de vertices: 303360
cadeira/cadeira.jpg RGB


In [15]:
modelo = load_model_from_file('mesa/mesa.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cube.obj. Vertice inicial:',len(vertices_list))
a = len(vertices_list)
faces_visited = []

for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], ' vertice inicial =', len(vertices_list))
        faces_visited.append(face[2])
    
    vertices = [modelo['vertices'][i - 1] for i in face[0]]
    textures = [modelo['texture'][i - 1] for i in face[1]]

    if len(face[0]) == 3:  # Triângulo
        vertices_list.extend(vertices)
        textures_coord_list.extend(textures)
    elif len(face[0]) == 4:  # Quadrilátero
        # Divisão em dois triângulos
        vertices_list.extend([vertices[0], vertices[1], vertices[2], vertices[0], vertices[2], vertices[3]])
        textures_coord_list.extend([textures[0], textures[1], textures[2], textures[0], textures[2], textures[3]])


print('total de vertices:',len(vertices_list)-a)

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(8,'mesa/mesa.jpg')


Processando modelo cube.obj. Vertice inicial: 910860
01___Default  vertice inicial = 910860
total de vertices: 234432
mesa/mesa.jpg RGB


In [16]:
modelo = load_model_from_file('mercurio/mercurio.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cube.obj. Vertice inicial:',len(vertices_list))
a = len(vertices_list)
faces_visited = []

for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], ' vertice inicial =', len(vertices_list))
        faces_visited.append(face[2])
    
    vertices = [modelo['vertices'][i - 1] for i in face[0]]
    textures = [modelo['texture'][i - 1] for i in face[1]]

    if len(face[0]) == 3:  # Triângulo
        vertices_list.extend(vertices)
        textures_coord_list.extend(textures)
    elif len(face[0]) == 4:  # Quadrilátero
        # Divisão em dois triângulos
        vertices_list.extend([vertices[0], vertices[1], vertices[2], vertices[0], vertices[2], vertices[3]])
        textures_coord_list.extend([textures[0], textures[1], textures[2], textures[0], textures[2], textures[3]])


print('total de vertices:',len(vertices_list)-a)

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(9,'mercurio/mercurio.jpg')


Processando modelo cube.obj. Vertice inicial: 1145292
Mercury  vertice inicial = 1145292
total de vertices: 47616
mercurio/mercurio.jpg RGB


In [17]:
modelo = load_model_from_file('venus/venus.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cube.obj. Vertice inicial:',len(vertices_list))
a = len(vertices_list)
faces_visited = []

for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], ' vertice inicial =', len(vertices_list))
        faces_visited.append(face[2])
    
    vertices = [modelo['vertices'][i - 1] for i in face[0]]
    textures = [modelo['texture'][i - 1] for i in face[1]]

    if len(face[0]) == 3:  # Triângulo
        vertices_list.extend(vertices)
        textures_coord_list.extend(textures)
    elif len(face[0]) == 4:  # Quadrilátero
        # Divisão em dois triângulos
        vertices_list.extend([vertices[0], vertices[1], vertices[2], vertices[0], vertices[2], vertices[3]])
        textures_coord_list.extend([textures[0], textures[1], textures[2], textures[0], textures[2], textures[3]])


print('total de vertices:',len(vertices_list)-a)

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(10,'venus/venus.jpg')


Processando modelo cube.obj. Vertice inicial: 1192908
Venus  vertice inicial = 1192908
total de vertices: 47616
venus/venus.jpg RGB


In [18]:
modelo = load_model_from_file('venus/venus.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cube.obj. Vertice inicial:',len(vertices_list))
a = len(vertices_list)
faces_visited = []

for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], ' vertice inicial =', len(vertices_list))
        faces_visited.append(face[2])
    
    vertices = [modelo['vertices'][i - 1] for i in face[0]]
    textures = [modelo['texture'][i - 1] for i in face[1]]

    if len(face[0]) == 3:  # Triângulo
        vertices_list.extend(vertices)
        textures_coord_list.extend(textures)
    elif len(face[0]) == 4:  # Quadrilátero
        # Divisão em dois triângulos
        vertices_list.extend([vertices[0], vertices[1], vertices[2], vertices[0], vertices[2], vertices[3]])
        textures_coord_list.extend([textures[0], textures[1], textures[2], textures[0], textures[2], textures[3]])


print('total de vertices:',len(vertices_list)-a)

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(10,'venus/venus.jpg')


Processando modelo cube.obj. Vertice inicial: 1240524
Venus  vertice inicial = 1240524
total de vertices: 47616
venus/venus.jpg RGB


In [19]:
modelo = load_model_from_file('marte/marte.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cube.obj. Vertice inicial:',len(vertices_list))
a = len(vertices_list)
faces_visited = []

for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], ' vertice inicial =', len(vertices_list))
        faces_visited.append(face[2])
    
    vertices = [modelo['vertices'][i - 1] for i in face[0]]
    textures = [modelo['texture'][i - 1] for i in face[1]]

    if len(face[0]) == 3:  # Triângulo
        vertices_list.extend(vertices)
        textures_coord_list.extend(textures)
    elif len(face[0]) == 4:  # Quadrilátero
        # Divisão em dois triângulos
        vertices_list.extend([vertices[0], vertices[1], vertices[2], vertices[0], vertices[2], vertices[3]])
        textures_coord_list.extend([textures[0], textures[1], textures[2], textures[0], textures[2], textures[3]])


print('total de vertices:',len(vertices_list)-a)

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(11,'marte/marte.jpg')


Processando modelo cube.obj. Vertice inicial: 1288140
Mars  vertice inicial = 1288140
total de vertices: 47616
marte/marte.jpg RGB


In [20]:
modelo = load_model_from_file('jupiter/jupiter.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cube.obj. Vertice inicial:',len(vertices_list))
a = len(vertices_list)
faces_visited = []

for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], ' vertice inicial =', len(vertices_list))
        faces_visited.append(face[2])
    
    vertices = [modelo['vertices'][i - 1] for i in face[0]]
    textures = [modelo['texture'][i - 1] for i in face[1]]

    if len(face[0]) == 3:  # Triângulo
        vertices_list.extend(vertices)
        textures_coord_list.extend(textures)
    elif len(face[0]) == 4:  # Quadrilátero
        # Divisão em dois triângulos
        vertices_list.extend([vertices[0], vertices[1], vertices[2], vertices[0], vertices[2], vertices[3]])
        textures_coord_list.extend([textures[0], textures[1], textures[2], textures[0], textures[2], textures[3]])


print('total de vertices:',len(vertices_list)-a)

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(12,'jupiter/jupiter.jpg')


Processando modelo cube.obj. Vertice inicial: 1335756
Jupiter  vertice inicial = 1335756
total de vertices: 47616
jupiter/jupiter.jpg RGB


In [21]:
modelo = load_model_from_file('saturno/saturno.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cube.obj. Vertice inicial:',len(vertices_list))
a = len(vertices_list)
faces_visited = []

for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], ' vertice inicial =', len(vertices_list))
        faces_visited.append(face[2])
    
    vertices = [modelo['vertices'][i - 1] for i in face[0]]
    textures = [modelo['texture'][i - 1] for i in face[1]]

    if len(face[0]) == 3:  # Triângulo
        vertices_list.extend(vertices)
        textures_coord_list.extend(textures)
    elif len(face[0]) == 4:  # Quadrilátero
        # Divisão em dois triângulos
        vertices_list.extend([vertices[0], vertices[1], vertices[2], vertices[0], vertices[2], vertices[3]])
        textures_coord_list.extend([textures[0], textures[1], textures[2], textures[0], textures[2], textures[3]])


print('total de vertices:',len(vertices_list)-a)

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(13,'saturno/saturno.jpg')


Processando modelo cube.obj. Vertice inicial: 1383372
13906_Saturn  vertice inicial = 1383372
total de vertices: 69120
saturno/saturno.jpg RGB


In [22]:
modelo = load_model_from_file('urano/urano.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cube.obj. Vertice inicial:',len(vertices_list))
a = len(vertices_list)
faces_visited = []

for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], ' vertice inicial =', len(vertices_list))
        faces_visited.append(face[2])
    
    vertices = [modelo['vertices'][i - 1] for i in face[0]]
    textures = [modelo['texture'][i - 1] for i in face[1]]

    if len(face[0]) == 3:  # Triângulo
        vertices_list.extend(vertices)
        textures_coord_list.extend(textures)
    elif len(face[0]) == 4:  # Quadrilátero
        # Divisão em dois triângulos
        vertices_list.extend([vertices[0], vertices[1], vertices[2], vertices[0], vertices[2], vertices[3]])
        textures_coord_list.extend([textures[0], textures[1], textures[2], textures[0], textures[2], textures[3]])


print('total de vertices:',len(vertices_list)-a)

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(14,'urano/urano_planeta.jpg')
load_texture_from_file(15,'urano/urano_anel.jpg')


Processando modelo cube.obj. Vertice inicial: 1452492
13907_Uranus_rings  vertice inicial = 1452492
13907_Uranus_planet  vertice inicial = 1566540
total de vertices: 161664
urano/urano_planeta.jpg RGB
urano/urano_anel.jpg RGB


In [23]:
modelo = load_model_from_file('netuno/netuno.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cube.obj. Vertice inicial:',len(vertices_list))
a = len(vertices_list)
faces_visited = []

for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], ' vertice inicial =', len(vertices_list))
        faces_visited.append(face[2])
    
    vertices = [modelo['vertices'][i - 1] for i in face[0]]
    textures = [modelo['texture'][i - 1] for i in face[1]]

    if len(face[0]) == 3:  # Triângulo
        vertices_list.extend(vertices)
        textures_coord_list.extend(textures)
    elif len(face[0]) == 4:  # Quadrilátero
        # Divisão em dois triângulos
        vertices_list.extend([vertices[0], vertices[1], vertices[2], vertices[0], vertices[2], vertices[3]])
        textures_coord_list.extend([textures[0], textures[1], textures[2], textures[0], textures[2], textures[3]])


print('total de vertices:',len(vertices_list)-a)

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(16,'netuno/netuno.jpg')


Processando modelo cube.obj. Vertice inicial: 1614156
13908_Neptune_ring  vertice inicial = 1614156
13908_Neptune_planet  vertice inicial = 1652172
total de vertices: 85632
netuno/netuno.jpg RGB


###  Enviando coordenadas para a GPU

In [24]:
# Request a buffer slot from GPU
buffer = glGenBuffers(2)


vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = textures_coord_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)






### Desenhando nossos modelos
* Cada modelo tem um Model para posicioná-los no mundo.
* É necessário saber qual a posição inicial e total de vértices de cada modelo
* É necessário indicar qual o ID da textura do modelo


In [25]:
# indice0      qtd_vertices    nome_obj   
# 0	           18720	       lua
# 18720	       36	       spiderman
# 468702	   325632	       nave
# 794334	   11904	       terra
# 806238	   47616	       sol

In [26]:

def desenha_lua(posicao_terra,t):
    # aplica a matriz model
    
    rot_x = rotacao_x(0)
    rot_y = rotacao_y(2* t)
    rot_z = rotacao_z(0)
    # translacao
    x = posicao_terra[0]
    y = posicao_terra[1]
    z = posicao_terra[2]
    # tr1 = translacao(-x,-y,-z)
    tr2 = translacao(x,y,z)

    # escala
    
    # tr2 = translacao(18, 0, 0)

    # escala
    es = escala(0.8, 0.8, 0.8)

    mat_model = model( [tr2, rot_x, rot_y, rot_z, es] )
    # mat_model = model( [ tr2] )
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 0)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 0, 18720) ## renderizando
    

    

In [27]:
 

def desenha_caixa(t):
    
    
    # aplica a matriz model
    
    # # rotacao
    # rot_x = rotacao_x(r_x)
    # rot_y = rotacao_y(r_y)
    # rot_z = rotacao_z(r_z)
    # translacao
    tr = translacao(0, 0, 0)

    # escala
    es = escala(150, 150, 150)
    

    mat_model = model( [tr, es] )
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 1)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 18720 , 36) ## renderizando
    

    

In [28]:
 

def desenha_nave(posicao_nave, t):
    rot_x = rotacao_x(90)
    # rot_y = rotacao_y(r_y)
    # rot_z = rotacao_z(r_z)
    # translacao
    x= posicao_nave[0]
    y= posicao_nave[1]
    z= posicao_nave[2]

    tr = translacao(x, y, z)

    # escala
    es = escala(1/15, 1/15.0, 1/15.0)

    mat_model = model([ tr, es, rot_x])
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 2)
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 18720+36 , 325632) ## renderizando
    

In [29]:
 

def desenha_terra(posicao_sol, t):
    
    
    # aplica a matriz model
    
    rot_x = rotacao_x(0)
    rot_y = rotacao_y(t*0.1)
    rot_z = rotacao_z(0)
    # translacao
    x = posicao_sol[0]
    y = posicao_sol[1]
    z = posicao_sol[2]

    tr = translacao(x,y,z)
    tr2 = translacao(50,0,0)
    # escala
    es = escala(0.25, 0.25, 0.25)


    mat_model = model([ tr,rot_x, rot_y, rot_z,tr2, es ])
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 3)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 18720+36+325632 , 11904) ## renderizando
    

    

In [30]:
 

def desenha_sol(t):
    
    
    # aplica a matriz model
    
    rot_x = rotacao_x(0)
    rot_y = rotacao_y(t*0.05)
    rot_z = rotacao_z(45)
    # translacao

    tr = translacao(0,0,0)
    # escala
    es = escala(1/30.0, 1/30.0, 1/30.0)


    mat_model = model([ tr,rot_x, rot_y, rot_z, es])
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 4)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 18720+36+325632+11904 , 47616) ## renderizando
    

    

In [31]:
 

def desenha_volante(posicao_nave,t):
    
    
    # aplica a matriz model
    
    rot_x = rotacao_x(90+10)
    rot_y = rotacao_y(180)
    rot_z = rotacao_z(180)
    # translacao
    x = posicao_nave[0]
    y = posicao_nave[1]
    z = posicao_nave[2]
    
    tr = translacao(x,y,z)
    tr2 = translacao(-0.01,0.2,-7)
    # escala
    es = escala(1/33.0, 1/33.0, 1/33.0)


    mat_model = model([ tr2,tr,rot_x, rot_y, rot_z, es])
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 5)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 18720+36+325632+11904+47616 , 176328) ## renderizando
    

    

In [32]:
 

def desenha_cama(posicao_nave,t):
    
    
    # aplica a matriz model
    
    rot_x = rotacao_x(90)
    rot_y = rotacao_y(180)
    rot_z = rotacao_z(180)
    # translacao
    x = posicao_nave[0]
    y = posicao_nave[1]
    z = posicao_nave[2]
    
    tr = translacao(x,y,z) #da nave
    tr2 = translacao(-0.15/3+0.05,-0.7,3) #posicao relativa à nave
    # escala
    es = escala(1/400.0, 1/400.0, 1/400.0)


    mat_model = model([ tr2,tr,rot_x, rot_y, rot_z, es])
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 6)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 18720+36+325632+11904+47616+176328 , 27264) ## renderizando
    

    

In [33]:
 

def desenha_cadeira2(posicao_nave,t):
    
    
    # aplica a matriz model
    
    rot_x = rotacao_x(90)
    rot_y = rotacao_y(180)
    rot_z = rotacao_z(0)
    # translacao
    x = posicao_nave[0]
    y = posicao_nave[1]
    z = posicao_nave[2]
    
    tr = translacao(x,y,z) #da nave
    tr2 = translacao(-0.15/3+0.05,-0.65, 1+(1/3)) #posicao relativa à nave
    # escala
    es = escala(1/100.0, 1/100.0, 1/100.0)


    mat_model = model([ tr2,tr,rot_x, rot_y, rot_z, es])
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 7)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 18720+36+325632+11904+47616+176328+27264 , 303360) ## renderizando
    

     

def desenha_cadeira(posicao_nave,t):
    
    
    # aplica a matriz model
    
    rot_x = rotacao_x(90)
    rot_y = rotacao_y(180)
    rot_z = rotacao_z(180)
    # translacao
    x = posicao_nave[0]
    y = posicao_nave[1]
    z = posicao_nave[2]
    
    tr = translacao(x,y,z) #da nave
    tr2 = translacao(-0.15/3+0.05,-0.65, 1/2) #posicao relativa à nave
    # escala
    es = escala(1/100.0, 1/100.0, 1/100.0)


    mat_model = model([ tr2,tr,rot_x, rot_y, rot_z, es])
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 7)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 18720+36+325632+11904+47616+176328+27264 , 303360) ## renderizando
    

    

In [34]:
 

def desenha_mesa(posicao_nave,t):
    
    
    # aplica a matriz model
    
    rot_x = rotacao_x(90)
    rot_y = rotacao_y(180)
    rot_z = rotacao_z(180)
    # translacao
    x = posicao_nave[0]
    y = posicao_nave[1]
    z = posicao_nave[2]
    
    tr = translacao(x,y,z) #da nave
    tr2 = translacao(-0.15/3+0.05,-0.65,1) #posicao relativa à nave
    # escala
    es = escala(1/100.0, 1/100.0, 1/100.0)


    mat_model = model([ tr2,tr,rot_x, rot_y, rot_z, es])
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 8)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 18720+36+325632+11904+47616+176328+27264+303360 , 234432) ## renderizando
    

    

In [35]:
 

def desenha_mercurio(posicao_sol, t):
    
    
    # aplica a matriz model
    
    rot_x = rotacao_x(0)
    rot_y = rotacao_y(t*0.1*4.15)
    rot_z = rotacao_z(0)
    # translacao
    x = posicao_sol[0]
    y = posicao_sol[1]
    z = posicao_sol[2]
    tr = translacao(x,y,z)
    tr2 = translacao(20,0,0)

    # escala
    es = escala(0.0025,0.0025,0.0025)


    mat_model = model([ tr,rot_x, rot_y, rot_z,tr2, es])
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 9)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 18720+36+325632+11904+47616+176328+27264+303360+234432 , 47616) ## renderizando
    

    

In [36]:
 

def desenha_venus(posicao_sol, t):
    
    
    # aplica a matriz model
    
    rot_x = rotacao_x(0)
    rot_y = rotacao_y(-t*0.1*1.63)
    rot_z = rotacao_z(0)
    # translacao
    x = posicao_sol[0]
    y = posicao_sol[1]
    z = posicao_sol[2]
    tr = translacao(x,y,z)
    tr2 = translacao(35,0,0)

    # escala
    es = escala(0.005,0.005,0.005)


    mat_model = model([ tr,rot_x, rot_y, rot_z,tr2, es])
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 10)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 18720+36+325632+11904+47616+176328+27264+303360+234432+47616 , 47616) ## renderizando
    

    

In [37]:
 

def desenha_marte(posicao_sol, t):
    
    
    # aplica a matriz model
    
    rot_x = rotacao_x(0)
    rot_y = rotacao_y(t*0.1*0.53)
    rot_z = rotacao_z(0)
    # translacao
    x = posicao_sol[0]
    y = posicao_sol[1]
    z = posicao_sol[2]
    tr = translacao(x,y,z)
    tr2 = translacao(60,0,0)

    # escala
    es = escala(0.0033,0.0033,0.0033)


    mat_model = model([ tr,rot_x, rot_y, rot_z,tr2, es])
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 11)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 18720+36+325632+11904+47616+176328+27264+303360+234432+47616+47616 , 47616) ## renderizando
    

    

In [38]:
 

def desenha_jupiter(posicao_sol, t):
    
    
    # aplica a matriz model
    
    rot_x = rotacao_x(0)
    rot_y = rotacao_y(t*0.1*0.08)
    rot_z = rotacao_z(0)
    # translacao
    x = posicao_sol[0]
    y = posicao_sol[1]
    z = posicao_sol[2]
    tr = translacao(x,y,z)
    tr2 = translacao(80,0,0)

    # escala
    es = escala(0.01,0.01,0.01)


    mat_model = model([ tr,rot_x, rot_y, rot_z,tr2, es])
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 12)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 18720+36+325632+11904+47616+176328+27264+303360+234432+47616+47616+47616 , 47616) ## renderizando
    

    

In [39]:
 

def desenha_saturno(posicao_sol, t):
    
    
    # aplica a matriz model
    
    rot_x = rotacao_x(0)
    rot_y = rotacao_y(t*0.1*0.03)
    rot_z = rotacao_z(0)
    # translacao
    x = posicao_sol[0]
    y = posicao_sol[1]
    z = posicao_sol[2]
    tr = translacao(x,y,z)
    tr2 = translacao(90,0,0)

    # escala
    es = escala(0.01,0.01,0.01)


    mat_model = model([ tr,rot_x, rot_y, rot_z,tr2, es])
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 13)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 18720+36+325632+11904+47616+176328+27264+303360+234432+47616+47616+47616 , 69120) ## renderizando
    

    

In [40]:
 

def desenha_urano(posicao_sol, t):
    
    
    # aplica a matriz model
    
    rot_x = rotacao_x(0)
    rot_y = rotacao_y(-t*0.1*0.01)
    rot_z = rotacao_z(0)
    # translacao
    x = posicao_sol[0]
    y = posicao_sol[1]
    z = posicao_sol[2]
    tr = translacao(x,y,z)
    tr2 = translacao(100,0,0)

    # escala
    es = escala(0.005,0.005,0.005)


    mat_model = model([ tr,rot_x, rot_y, rot_z,tr2, es])
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 14)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 18720+36+325632+11904+47616+176328+27264+303360+234432+47616+47616+47616+69120 , 114048) ## renderizando
    
    glBindTexture(GL_TEXTURE_2D, 15)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 18720+36+325632+11904+47616+176328+27264+303360+234432+47616+47616+47616+69120+114048 , 161664) ## renderizando
    
    

In [41]:
 

def desenha_netuno(posicao_sol, t):
    
    
    # aplica a matriz model
    
    rot_x = rotacao_x(0)
    rot_y = rotacao_y(t*0.1*0.006)
    rot_z = rotacao_z(0)
    # translacao
    x = posicao_sol[0]
    y = posicao_sol[1]
    z = posicao_sol[2]
    tr = translacao(x,y,z)
    tr2 = translacao(110,0,0)

    # escala
    es = escala(0.004,0.004,0.004)


    mat_model = model([ tr,rot_x, rot_y, rot_z,tr2, es])
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_FALSE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 16)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 18720+36+325632+11904+47616+176328+27264+303360+234432+47616+47616+47616+69120+114048+161664 , 38016) ## renderizando
    
    glDrawArrays(GL_TRIANGLES, 18720+36+325632+11904+47616+176328+27264+303360+234432+47616+47616+47616+69120+114048+161664+38016 , 85632) ## renderizando
    

### Eventos para modificar a posição da câmera.

* Usei as teclas A, S, D e W para movimentação no espaço tridimensional
* Usei a posição do mouse para "direcionar" a câmera

In [42]:
cameraPos   = glm.vec3(0.0,  0.0,  15.0);
cameraFront = glm.vec3(0.0,  0.0, 0.0);
cameraUp    = glm.vec3(0.0,  1.0,  0.0);


polygonal_mode = False


inc_fov = 0
inc_near = 0
inc_far = 0
inc_view_up = 0

def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp, polygonal_mode, inc_fov, inc_near, inc_far, cameraUp, inc_view_up, x_nav, y_nav
     
    if key == 66:
        inc_view_up += 0.1
    if key == 78: inc_near += 0.1
    if key == 77: inc_far -= 5
        
    
    print(cameraPos)
    cameraSpeed = 0.25
    if key == 87 and (action==1 or action==2): # tecla W
        cameraPos += cameraSpeed * cameraFront
    
    if key == 83 and (action==1 or action==2): # tecla S
        cameraPos -= cameraSpeed * cameraFront
    
    if key == 65 and (action==1 or action==2): # tecla A
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 68 and (action==1 or action==2): # tecla D
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 80 and action==1 and polygonal_mode==True:
        polygonal_mode=False
    else:
        if key == 80 and action==1 and polygonal_mode==False:
            polygonal_mode=True
        
        
        
firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2

def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos

    sensitivity = 0.3 
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset;
    pitch += yoffset;

    
    if pitch >= 90.0: pitch = 90.0
    if pitch <= -90.0: pitch = -90.0

    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)


    
glfw.set_key_callback(window,key_event)
glfw.set_cursor_pos_callback(window, mouse_event)


### Matrizes Model, View e Projection

Teremos uma aula específica para entender o seu funcionamento.

In [43]:

def rotacao_x(angle):
    rad = math.radians(angle)
    matrix = [
        [1, 0, 0, 0],
        [0, math.cos(rad), -math.sin(rad), 0],
        [0, math.sin(rad), math.cos(rad), 0],
        [0, 0, 0, 1]
    ]
    return np.array(matrix)

def rotacao_y(angle):
    rad = math.radians(angle)
    matrix = [
        [math.cos(rad), 0, math.sin(rad), 0],
        [0, 1, 0, 0],
        [-math.sin(rad), 0, math.cos(rad), 0],
        [0, 0, 0, 1]
    ]
    return np.array(matrix)

def rotacao_z(angle):
    rad = math.radians(angle)
    matrix = [
        [math.cos(rad), -math.sin(rad), 0, 0],
        [math.sin(rad), math.cos(rad), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ]
    return np.array(matrix)

def escala(sx, sy, sz):
    matrix = [
        [sx, 0, 0, 0],
        [0, sy, 0, 0],
        [0, 0, sz, 0],
        [0, 0, 0, 1]
    ]
    return np.array(matrix)

def translacao(tx, ty, tz):
    matrix = [
        [1, 0, 0, tx],
        [0, 1, 0, ty],
        [0, 0, 1, tz],
        [0, 0, 0, 1]
    ]
    return np.array(matrix)

def multiplica_matrizes(m1, m2):
    return np.dot(m1, m2)

def multiplica_n_matrizes(lista_de_matrizes):
    result = lista_de_matrizes[0]
    for i in range(1, len(lista_de_matrizes)):
        result = multiplica_matrizes(result, lista_de_matrizes[i])
    return result


def model(lista_de_matrizes):
    
    matrix_transform = multiplica_n_matrizes(lista_de_matrizes)
    
    matrix_transform = np.array(matrix_transform).T # pegando a transposta da matriz (glm trabalha com ela invertida)
    
    return matrix_transform

def view():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura, inc_fov, inc_near, inc_far
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(45.0), largura/altura, 0.1, 1000.0)
    mat_projection = np.array(mat_projection)    
    return mat_projection

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

In [ ]:
glfw.show_window(window)
glfw.set_cursor_pos(window, lastX, lastY)
glEnable(GL_DEPTH_TEST) ### importante para 3D
   
incremento = 0
posicao_sol = np.array([0,0,0])
posicao_terra = posicao_sol+ np.array([30,0,0])
# posicao_lua = np.array([10,0,0])
posicao_nave = np.array([0,20,0])

while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    incremento+=1
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)  
    
    glClearColor(0.0, 0.0, 0.0, 1.0)
    
    if polygonal_mode==True:
        glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    if polygonal_mode==False:
        glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)
    
    

    # desenha_lua(posicao_terra,posicao_lua, incremento)   
    desenha_caixa(0)
    desenha_sol( incremento)
    desenha_mercurio(posicao_sol, incremento)
    desenha_venus(posicao_sol, incremento)
    desenha_terra(posicao_sol, incremento)
    desenha_marte(posicao_sol, incremento)
    desenha_jupiter(posicao_sol, incremento)
    desenha_saturno(posicao_sol, incremento)
    desenha_urano(posicao_sol, incremento)
    desenha_netuno(posicao_sol, incremento)

    desenha_nave(posicao_nave,incremento)
    
    desenha_volante(posicao_nave, incremento)
    desenha_cama(posicao_nave, incremento)
    desenha_cadeira(posicao_nave, incremento)
    desenha_cadeira2(posicao_nave, incremento)
    desenha_mesa(posicao_nave, incremento)


    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_TRUE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_TRUE, mat_projection)    
    
    

    
    glfw.swap_buffers(window)

glfw.terminate()